In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

# Path to the dataset folder containing subdirectories for each clothing category
dataset_path = '/content/drive/MyDrive/Colab Notebooks/FYP/Recommendation System/polyvore'
# Data preprocessing
datagen = ImageDataGenerator(rescale=1.0/255)

# Define image dimensions
img_height, img_width = 100, 100

# Load and preprocess images
image_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode=None,
    shuffle=False
)

# Create a list of image file paths
image_files = [os.path.join(dataset_path, fname) for fname in image_generator.filenames]

# Load and preprocess image features using a pre-trained model (e.g., VGG16)
pretrained_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False)

# Extract features from the pre-trained model
image_features = []
for image_path in image_files:
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = tf.keras.applications.vgg16.preprocess_input(x)
    features = pretrained_model.predict(x)
    image_features.append(features.flatten())

image_features = np.array(image_features)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 18ms/step


In [3]:
num_users = 100
num_items = len(image_files)
embedding_dim = 64

# Simulated user-item interactions (0 for no interaction, 1 for interaction)
interactions = np.random.choice([0, 1], size=(num_users, num_items))

# Add an additional input for occasions
num_occasions = 5  # You can change this based on the number of occasions
occasions_input = Input(shape=(1,), name='occasions_input')
occasion_embedding = Embedding(num_occasions, embedding_dim)(occasions_input)
occasion_vecs = Flatten()(occasion_embedding)

# Split data into train and test sets
train_size = 0.8
train_interactions, test_interactions = train_test_split(interactions, train_size=train_size, random_state=42)

# NCF Model
embedding_dim = 64
user_input = Input(shape=(1,), name='user_input')
item_input = Input(shape=(1,), name='item_input')
user_embedding = Embedding(num_users, embedding_dim)(user_input)
item_embedding = Embedding(num_items, embedding_dim)(item_input)
user_vecs = Flatten()(user_embedding)
item_vecs = Flatten()(item_embedding)

concat = Concatenate()([user_vecs, item_vecs, occasion_vecs])  # Include occasion vectors in the concatenation
dense1 = Dense(128, activation='relu')(concat)
dense2 = Dense(64, activation='relu')(dense1)
output = Dense(1, activation='sigmoid')(dense2)

model = Model(inputs=[user_input, item_input, occasions_input], outputs=output)  # Include the occasions input
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the NCF model
batch_size = 64
num_epochs = 55
model.fit([train_interactions[:, 0], train_interactions[:, 1], train_interactions[:, 2]],  # Include occasions
          train_interactions[:, 3],
          batch_size=batch_size, epochs=num_epochs, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate([test_interactions[:, 0], test_interactions[:, 1], test_interactions[:, 2]],  # Include occasions
                               test_interactions[:, 3])
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/55
2/2 [==============================] - 4s 42ms/step - loss: 0.6914 - accuracy: 0.5750
Epoch 2/55
2/2 [==============================] - 0s 9ms/step - loss: 0.6860 - accuracy: 0.6000
Epoch 3/55
2/2 [==============================] - 0s 9ms/step - loss: 0.6833 - accuracy: 0.5875
Epoch 4/55
2/2 [==============================] - 0s 8ms/step - loss: 0.6807 - accuracy: 0.5875
Epoch 5/55
2/2 [==============================] - 0s 11ms/step - loss: 0.6782 - accuracy: 0.5875
Epoch 6/55
2/2 [==============================] - 0s 9ms/step - loss: 0.6765 - accuracy: 0.5875
Epoch 7/55
2/2 [==============================] - 0s 8ms/step - loss: 0.6742 - accuracy: 0.5875
Epoch 8/55
2/2 [==============================] - 0s 7ms/step - loss: 0.6723 - accuracy: 0.5875
Epoch 9/55
2/2 [==============================] - 0s 8ms/step - loss: 0.6708 - accuracy: 0.5875
Epoch 10/55
2/2 [==============================] - 0s 7ms/step - loss: 0.6689 - accuracy: 0.5875
Epoch 11/55
2/2 [====================

In [4]:
from IPython.display import display, Image
import ipywidgets as widgets
import os

# Create a widget for selecting the occasion
occasion_dropdown = widgets.Dropdown(
    options=['casual', 'formal', 'party', 'business'],
    description='Select Occasion:'
)

# Directory where occasion-specific subdirectories are located
base_directory = '/content/drive/MyDrive/Colab Notebooks/FYP/Recommendation System/occasion/'

def get_outfit_recommendations(occasion):
    # Replace this with your recommendation logic
    # You should generate a list of outfit image file names based on the selected occasion
    recommendations = []

    # In this example, we'll use placeholders. Replace these with actual image file names.
    if occasion == 'casual':
        recommendations = ['5464842_1.jpg', '100118405_2.jpg']
    elif occasion == 'formal':
        recommendations = ['2941774_1.jpg', '10962342_2.jpg','108913270_2.jpg']
    elif occasion == 'party':
        recommendations = ['8637501_1.jpg']
    elif occasion == 'business':
        recommendations = ['100176564_1.jpg', '12785617_2.jpg']

    return recommendations

def recommend_outfit(occasion):
    # Get outfit recommendations based on the selected occasion
    outfit_recommendations = get_outfit_recommendations(occasion)

    if outfit_recommendations:
        print(f'Recommended outfits for {occasion}:')
        for outfit_filename in outfit_recommendations:
            # Construct the full path to the outfit image
            outfit_image_path = os.path.join(base_directory, occasion, outfit_filename)
            display(Image(filename=outfit_image_path))
    else:
        print(f'No recommendations available for {occasion}.')

# Bind the widget to the function
widgets.interactive(recommend_outfit, occasion=occasion_dropdown)


interactive(children=(Dropdown(description='Select Occasion:', options=('casual', 'formal', 'party', 'business…